In [16]:
import pandas as pd
from openpyxl import load_workbook

start_file = "TOP 10 LEADING MAY 2023.xlsx"
file = "Book2.xlsx"

wb = load_workbook(start_file)
ws = wb.active

#unmegring cells & removal of empty rows
for merged_range in list(ws.merged_cells.ranges):
    ws.unmerge_cells(str(merged_range))

wb.save("Book2.xlsx")

df = pd.read_excel("Book2.xlsx", header=None)
df = df.dropna(how="all").drop(df.columns[0], axis=1)

df.to_excel(file, index=False)


In [14]:
#adding header

df.insert(0, "NewColumn", "")

df.iloc[:, 1:] = df.iloc[:, 1:].shift(axis=1)
df.columns = ["Month_year", "Consultation_Type", "Case","Under 1 Male", "Under 1 Female", "1-4 Male", "1-4 Female"
              ,"5-9 Male", "5-9 Female", "10-14 Male", "10-14 Female", "15-18 Male","15-18 Female","19-24 Male", "19-24 Female",
              "25-29 Male", "25-29 Female", "30-34 Male", "30-34 Female", "35-39 Male", "35-39 Female", "40-44 Male", "40-44 Female",
              "45-49 Male", "45-49 Female", "50-54 Male", "50-54 Female", "55-59 Male", "55-59 Female", "60-64 Male", "60-64 Female",
              "65-69 Male", "65-69 Female", "70 Over Male", "70 Over Female"]
df.to_excel(file, index=False)


In [9]:
#adding date

wb = load_workbook("Book2.xlsx")
ws = wb.active

found_value = None

for row in ws.iter_rows():
    for cell in row:
        if cell.value and "MONTH AND YEAR:" in str(cell.value).strip().upper():
            text = str(cell.value).strip()
            found_value = text.split("MONTH AND YEAR:")[-1].strip()
            found_value
    
    if found_value:
        break

if found_value:
    for row in range(2, ws.max_row + 1):
        ws.cell(row=row, column=1, value=found_value)
    wb.save("Book2.xlsx")
    print(f"Added '{found_value}' to the first column for all rows.")
else:
    print("'MONTH AND YEAR:' not found in the sheet.")



Added 'MAY 2023' to the first column for all rows.


In [10]:
#delete 5 next row
df = pd.read_excel(file, sheet_name=0)

df = df.drop(df.index[0:8], axis=0)
df.to_excel(file, index=False)



In [11]:
#deleting unecessary text
#in the 2nd column in the fisrt 10 rows put a value of 1, in the next 10 rows add a value of 2 and in last remaning rows with value add 3
target = "PASIG CITY CHILDREN'S HOSPITAL/PASIG CITY COVID-19 REFERRAL CENTER"



idx = df.apply(lambda row: row.astype(str).str.contains(target, na=False)).any(axis=1)
target_indices = df.index[idx].tolist()

rows_to_delete = []
for start_idx in target_indices:
    end_idx = min(start_idx + 7, len(df) - 1) 
    rows_to_delete.extend(range(start_idx, end_idx + 1))

df = df[~df.astype(str).apply(lambda row: row.str.contains("total", case=False, na=False)).any(axis=1)]
df = df.drop(rows_to_delete).reset_index(drop=True)

#assigning case value and adding 0 to empty cells
n = len(df)
df.iloc[:min(10, n), 1] = 1
if n > 10:
    df.iloc[10:min(20, n), 1] = 2
if n > 20:
    df.iloc[20:, 1] = 3
df.to_excel(file, index=False)


df = df.fillna(0)

print(df.head())


  Month_year  Consultation_Type            Case  Under 1 Male  Under 1 Female  \
0   MAY 2023                1.0           FEVER            32              23   
1   MAY 2023                1.0        VOMITING            14               7   
2   MAY 2023                1.0  ABDOMINAL PAIN             1               0   
3   MAY 2023                1.0             DOB             2               3   
4   MAY 2023                1.0             LBM            15               7   

   1-4 Male  1-4 Female  5-9 Male  5-9 Female  10-14 Male  ...  50-54 Male  \
0        95          78        67          51          21  ...           0   
1        45          40        18          20           5  ...           1   
2        12           6        22          20           9  ...           5   
3         4           4         2           1           3  ...           7   
4        29          14         3           2           3  ...           0   

   50-54 Female  55-59 Male  55-59 Female  6

C:\Users\PLPASIG\AppData\Local\Temp\ipykernel_21604\386304020.py:28: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.fillna(0)
